In [1]:
# Chargement d'une petite base de textes
import loadFilesPartial as lfp
data,Y=lfp.loadLabeled("./data/train",10)
print len(data)

20


In [2]:
# Le item est nécessaire pour convertir au format Double (python natif) 
# car les dataframes semblent avoir des problèmes avec les types numpy
labeledData = zip(data,[y.item() for y in Y])

In [3]:
labeledRdd = sc.parallelize(labeledData)

In [4]:
from bs4 import BeautifulSoup as bfs

In [5]:
# Définition d'une fonction de nettoyage : on remplace les balises par des espaces
# à l'aide du parser de hmtl, et on met tout en minuscules.
def cleanLower(doc):
    clean = bfs(doc).get_text(separator=' ')
    return clean.lower()

In [6]:
# On applique le nettoyage à la RDD
cleanRdd = labeledRdd.map(lambda doc : (cleanLower(doc[0]),doc[1]))

In [7]:
# On va utiliser la librairie spark ML donc on travaille avec des dataframes
# EN THEORIE, c'est plus facile. C'est un mensonge.
df = sqlContext.createDataFrame(cleanRdd, ['review', 'label'])

In [8]:
dfTrain, dfTest = df.randomSplit([0.8,0.2])

In [9]:
dfTrain.show()

+--------------------+-----+
|              review|label|
+--------------------+-----+
|homelessness (or ...|  1.0|
|brilliant over-ac...|  1.0|
|this is easily th...|  1.0|
|this is not the t...|  1.0|
|in this "critical...|  1.0|
|the night listene...|  1.0|
|you know, robin w...|  1.0|
|airport '77 start...|  0.0|
|this film lacked ...|  0.0|
|sorry everyone,,,...|  0.0|
|the second attemp...|  0.0|
|i don't know who ...|  0.0|
|this film is medi...|  0.0|
|the film is bad. ...|  0.0|
+--------------------+-----+



In [10]:
dfTest.show()

+--------------------+-----+
|              review|label|
+--------------------+-----+
|bromwell high is ...|  1.0|
|this isn't the co...|  1.0|
|yes its an art......|  1.0|
|story of a man wh...|  0.0|
|when i was little...|  0.0|
|"it appears that ...|  0.0|
+--------------------+-----+



In [11]:
#Tokenizing
from pyspark.ml.feature import Tokenizer
tokenizer = Tokenizer(inputCol='review', outputCol='words')
dfTrainTok = tokenizer.transform(dfTrain)

In [39]:
# La ponctuation n'a pas l'air gérée très correctement...
dfTrainTok.select('words').take(1)

[Row(words=[u'homelessness', u'(or', u'houselessness', u'as', u'george', u'carlin', u'stated)', u'has', u'been', u'an', u'issue', u'for', u'years', u'but', u'never', u'a', u'plan', u'to', u'help', u'those', u'on', u'the', u'street', u'that', u'were', u'once', u'considered', u'human', u'who', u'did', u'everything', u'from', u'going', u'to', u'school,', u'work,', u'or', u'vote', u'for', u'the', u'matter.', u'most', u'people', u'think', u'of', u'the', u'homeless', u'as', u'just', u'a', u'lost', u'cause', u'while', u'worrying', u'about', u'things', u'such', u'as', u'racism,', u'the', u'war', u'on', u'iraq,', u'pressuring', u'kids', u'to', u'succeed,', u'technology,', u'the', u'elections,', u'inflation,', u'or', u'worrying', u'if', u"they'll", u'be', u'next', u'to', u'end', u'up', u'on', u'the', u'streets.', u'but', u'what', u'if', u'you', u'were', u'given', u'a', u'bet', u'to', u'live', u'on', u'the', u'streets', u'for', u'a', u'month', u'without', u'the', u'luxuries', u'you', u'once', u'h

In [13]:
# J'ai écrit cette fonction au cas où on souhaiterait absolument passer par des 
# pipelines, et intégrer les stopword dans la structure d'un transformer devait 
# passer par ce code grosso modo. Au final, on se passera surement des pipelines
# car leurs possibilités sont limitées sur spark ml. Très limitées. C'est de la 
# merde en fait. Voila.

import nltk
from pyspark.ml.pipeline import Transformer
from pyspark.ml.param.shared import HasInputCol, HasOutputCol, Param
from pyspark.ml.util import keyword_only

from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, StringType

class NLTKWordPunctTokenizer(Transformer, HasInputCol, HasOutputCol):

    @keyword_only
    def __init__(self, inputCol=None, outputCol=None, stopwords=None):
        super(NLTKWordPunctTokenizer, self).__init__()
        self.stopwords = Param(self, "stopwords", "")
        self._setDefault(stopwords=set())
        kwargs = self.__init__._input_kwargs
        self.setParams(**kwargs)

    @keyword_only
    def setParams(self, inputCol=None, outputCol=None, stopwords=None):
        kwargs = self.setParams._input_kwargs
        return self._set(**kwargs)

    def setStopwords(self, value):
        self._paramMap[self.stopwords] = value
        return self

    def getStopwords(self):
        return self.getOrDefault(self.stopwords)

    def _transform(self, dataset):
        stopwords = self.getStopwords()

        def f(s):
            tokens = nltk.tokenize.wordpunct_tokenize(s)
            return [t for t in tokens if t.lower() not in stopwords]

        t = ArrayType(StringType())
        out_col = self.getOutputCol()
        in_col = dataset[self.getInputCol()]
        return dataset.withColumn(out_col, udf(f, t)(in_col))

In [14]:
# Par contre cette fonction marche bien
tokenizerNoSw = NLTKWordPunctTokenizer(
    inputCol="review", outputCol="wordsNoSw",  
    stopwords=set(nltk.corpus.stopwords.words('english')))

In [15]:
dfTrainTokNoSw = tokenizerNoSw.transform(dfTrainTok)

In [16]:
dfTrainTokNoSw.show()

+--------------------+-----+--------------------+--------------------+
|              review|label|               words|           wordsNoSw|
+--------------------+-----+--------------------+--------------------+
|homelessness (or ...|  1.0|[homelessness, (o...|[homelessness, (,...|
|brilliant over-ac...|  1.0|[brilliant, over-...|[brilliant, -, ac...|
|this is easily th...|  1.0|[this, is, easily...|[easily, underrat...|
|this is not the t...|  1.0|[this, is, not, t...|[typical, mel, br...|
|in this "critical...|  1.0|[in, this, "criti...|[", critically, a...|
|the night listene...|  1.0|[the, night, list...|[night, listener,...|
|you know, robin w...|  1.0|[you, know,, robi...|[know, ,, robin, ...|
|airport '77 start...|  0.0|[airport, '77, st...|[airport, ', 77, ...|
|this film lacked ...|  0.0|[this, film, lack...|[film, lacked, so...|
|sorry everyone,,,...|  0.0|[sorry, everyone,...|[sorry, everyone,...|
|the second attemp...|  0.0|[the, second, att...|[second, attempt,...|
|i don

In [40]:
# La ponctuation est bien gérée
dfTrainTokNoSw.select('wordsNoSw').take(1)

[Row(wordsNoSw=[u'homelessness', u'(', u'houselessness', u'george', u'carlin', u'stated', u')', u'issue', u'years', u'never', u'plan', u'help', u'street', u'considered', u'human', u'everything', u'going', u'school', u',', u'work', u',', u'vote', u'matter', u'.', u'people', u'think', u'homeless', u'lost', u'cause', u'worrying', u'things', u'racism', u',', u'war', u'iraq', u',', u'pressuring', u'kids', u'succeed', u',', u'technology', u',', u'elections', u',', u'inflation', u',', u'worrying', u"'", u'll', u'next', u'end', u'streets', u'.', u'given', u'bet', u'live', u'streets', u'month', u'without', u'luxuries', u'home', u',', u'entertainment', u'sets', u',', u'bathroom', u',', u'pictures', u'wall', u',', u'computer', u',', u'everything', u'treasure', u'see', u"'", u'like', u'homeless', u'?', u'goddard', u'bolt', u"'", u'lesson', u'.', u'mel', u'brooks', u'(', u'directs', u')', u'stars', u'bolt', u'plays', u'rich', u'man', u'everything', u'world', u'deciding', u'make', u'bet', u'sissy', 

In [25]:
from pyspark.sql.types import StructType, StructField

class NLTKPosTagger(Transformer, HasInputCol, HasOutputCol):

    @keyword_only
    def __init__(self, inputCol=None, outputCol=None, tagset=None):
        super(NLTKPosTagger, self).__init__()
        self.tagset = Param(self, "tagset", "")
        self._setDefault(tagset='universal')
        kwargs = self.__init__._input_kwargs
        self.setParams(**kwargs)

    @keyword_only
    def setParams(self, inputCol=None, outputCol=None, tagset=None):
        kwargs = self.setParams._input_kwargs
        return self._set(**kwargs)

    def setTagset(self, value):
        self._paramMap[self.tagset] = value
        return self

    def getTagset(self):
        return self.getOrDefault(self.tagset)

    def _transform(self, dataset):
        tagset = self.getTagset()

        def f(s):
            return nltk.pos_tag(s,tagset=tagset)

        fields = (StructField('word',StringType(),True),StructField('tag',StringType(),True))
        t = ArrayType(StructType(fields))
        out_col = self.getOutputCol()
        in_col = dataset[self.getInputCol()]
        return dataset.withColumn(out_col, udf(f, t)(in_col))

In [26]:
posTagger = NLTKPosTagger(
    inputCol="words", outputCol="tagWords")

In [27]:
dfTagged = posTagger.transform(dfTrainTok)

In [28]:
dfTagged.show()

+--------------------+-----+--------------------+--------------------+
|              review|label|               words|            tagWords|
+--------------------+-----+--------------------+--------------------+
|homelessness (or ...|  1.0|[homelessness, (o...|[[homelessness,NO...|
|brilliant over-ac...|  1.0|[brilliant, over-...|[[brilliant,ADJ],...|
|this is easily th...|  1.0|[this, is, easily...|[[this,DET], [is,...|
|this is not the t...|  1.0|[this, is, not, t...|[[this,DET], [is,...|
|in this "critical...|  1.0|[in, this, "criti...|[[in,ADP], [this,...|
|the night listene...|  1.0|[the, night, list...|[[the,DET], [nigh...|
|you know, robin w...|  1.0|[you, know,, robi...|[[you,PRON], [kno...|
|airport '77 start...|  0.0|[airport, '77, st...|[[airport,NOUN], ...|
|this film lacked ...|  0.0|[this, film, lack...|[[this,DET], [fil...|
|sorry everyone,,,...|  0.0|[sorry, everyone,...|[[sorry,NOUN], [e...|
|the second attemp...|  0.0|[the, second, att...|[[the,DET], [seco...|
|i don

In [29]:
from pyspark.ml.feature import NGram
bigram = NGram(inputCol="words", outputCol="bigrams")
dfBigram = bigram.transform(dfTrainTokNoSw)

In [30]:
dfBigram.show()

+--------------------+-----+--------------------+--------------------+--------------------+
|              review|label|               words|           wordsNoSw|             bigrams|
+--------------------+-----+--------------------+--------------------+--------------------+
|homelessness (or ...|  1.0|[homelessness, (o...|[homelessness, (,...|[homelessness (or...|
|brilliant over-ac...|  1.0|[brilliant, over-...|[brilliant, -, ac...|[brilliant over-a...|
|this is easily th...|  1.0|[this, is, easily...|[easily, underrat...|[this is, is easi...|
|this is not the t...|  1.0|[this, is, not, t...|[typical, mel, br...|[this is, is not,...|
|in this "critical...|  1.0|[in, this, "criti...|[", critically, a...|[in this, this "c...|
|the night listene...|  1.0|[the, night, list...|[night, listener,...|[the night, night...|
|you know, robin w...|  1.0|[you, know,, robi...|[know, ,, robin, ...|[you know,, know,...|
|airport '77 start...|  0.0|[airport, '77, st...|[airport, ', 77, ...|[airport '

In [31]:
# Pour la suite on a le choix entre l'encodage utilisé par le prof (le mot y est ou n'y est pas)
# ou la version en apparence plus informative du tfidf. En vrai, le tfidf peut être trompeur 
# donc je construis quand même les dictionnaires d'unigrammes et de bigrammes pour pouvoir 
# calculer les sparse vectors du prof.

In [32]:
import itertools
lists=dfBigram.map(lambda r : r.words).collect()
dictUnigrams=set(itertools.chain(*lists))

In [33]:
lists2=dfBigram.map(lambda r : r.bigrams).collect()
dictBigrams=set(itertools.chain(*lists2))

In [34]:
dictionaryUni={}
for i,word in enumerate(dictUnigrams):
	dictionaryUni[word]=i

In [35]:
dictionaryBigrams={}
for i,word in enumerate(dictBigrams):
	dictionaryBigrams[word]=i

In [41]:
'!' in dictionaryUni

False

In [42]:
# Voila qui est mieux...
lists3=dfBigram.map(lambda r : r.wordsNoSw).collect()
dict3=set(itertools.chain(*lists3))
dictionary3 = {}
for i,word in enumerate(dict3):
    dictionary3[word]=i
'!' in dictionary3

True

In [48]:
from string import punctuation
import re
r = re.compile(r'[\s{}]+'.format(re.escape(punctuation)))

In [49]:
for k in dict3:
    m = r.search(k)
    if m:
        print m.group()

;
&
/
,,
?)
,"
?"
!
").
,
'
"
":
*******
-
'."
**
(
."
))
),
).
.
~
)
",
".
"?
...
("
?
,,,
!)
:
.")


In [51]:
# Fonction calculant le sparse vector correspondant à un ensemble de tokens
from pyspark.mllib.linalg import SparseVector
def vectorizeUni(tokens):
    vector_dict={}
    for w in tokens:
        vector_dict[dictionaryUni[w]]=1
    return SparseVector(len(dictionaryUni),vector_dict)

def vectorizeBi(tokens):
    vector_dict={}
    for w in tokens:
        vector_dict[dictionaryBigrams[w]]=1
    return SparseVector(len(dictionaryBigrams),vector_dict)

In [52]:
# La ca devient le bordel, j'en ai chié pour arriver à appliquer une fonction à toute une colonne 
# d'une dataframe. Contrairement à pandas, y a pas de fonction "apply", il faut recourir à des 
# UserDefinedFunctions, et penser que le type sparseVector ne sera pas reconnu par la dataframe, qui
# n'est compatible qu'avec un nombre restreint de types

# EDIT : en fait je m'en suis pas rendu compte, mais cette manip je l'avais déjà faite pour la surcharge des
# tokenizer et postagger... les cinq dernières lignes à la fin avec udf et tout

In [53]:
from pyspark.sql.functions import UserDefinedFunction

In [54]:
from pyspark.mllib.linalg import VectorUDT

In [55]:
udfVectorizeUni=UserDefinedFunction(lambda x : vectorizeUni(x),
                                    VectorUDT())

In [56]:
# Une dataframe est un objet immutable, donc pas la peine d'essayer de modifier une colonne,
# à la place on crée une deuxième dataframe où on ajoute la colonne qu'on veut.
dfVect = dfBigram.withColumn("words", udfVectorizeUni("words"))

In [57]:
# On a bien remplacé ici du coup les mots par les vecteurs sparse
dfVect.show()

+--------------------+-----+--------------------+--------------------+--------------------+
|              review|label|               words|           wordsNoSw|             bigrams|
+--------------------+-----+--------------------+--------------------+--------------------+
|homelessness (or ...|  1.0|(1573,[14,26,28,3...|[homelessness, (,...|[homelessness (or...|
|brilliant over-ac...|  1.0|(1573,[4,22,25,30...|[brilliant, -, ac...|[brilliant over-a...|
|this is easily th...|  1.0|(1573,[37,71,85,8...|[easily, underrat...|[this is, is easi...|
|this is not the t...|  1.0|(1573,[10,37,61,6...|[typical, mel, br...|[this is, is not,...|
|in this "critical...|  1.0|(1573,[0,12,20,21...|[", critically, a...|[in this, this "c...|
|the night listene...|  1.0|(1573,[0,6,16,33,...|[night, listener,...|[the night, night...|
|you know, robin w...|  1.0|(1573,[3,23,29,36...|[know, ,, robin, ...|[you know,, know,...|
|airport '77 start...|  0.0|(1573,[7,9,11,13,...|[airport, ', 77, ...|[airport '

In [58]:
udfVectorizeBi=UserDefinedFunction(lambda x : vectorizeBi(x),
                                   VectorUDT())

In [59]:
dfVect2 = dfVect.withColumn("bigrams", udfVectorizeBi("bigrams"))

In [60]:
dfVect2.show()

+--------------------+-----+--------------------+--------------------+--------------------+
|              review|label|               words|           wordsNoSw|             bigrams|
+--------------------+-----+--------------------+--------------------+--------------------+
|homelessness (or ...|  1.0|(1573,[14,26,28,3...|[homelessness, (,...|(3370,[14,16,25,3...|
|brilliant over-ac...|  1.0|(1573,[4,22,25,30...|[brilliant, -, ac...|(3370,[7,30,33,40...|
|this is easily th...|  1.0|(1573,[37,71,85,8...|[easily, underrat...|(3370,[6,101,124,...|
|this is not the t...|  1.0|(1573,[10,37,61,6...|[typical, mel, br...|(3370,[4,21,84,98...|
|in this "critical...|  1.0|(1573,[0,12,20,21...|[", critically, a...|(3370,[0,17,26,36...|
|the night listene...|  1.0|(1573,[0,6,16,33,...|[night, listener,...|(3370,[9,35,38,50...|
|you know, robin w...|  1.0|(1573,[3,23,29,36...|[know, ,, robin, ...|(3370,[1,52,54,58...|
|airport '77 start...|  0.0|(1573,[7,9,11,13,...|[airport, ', 77, ...|(3370,[2,6

In [61]:
# Pour les opérations de traitement du langage, il est d'usage de normaliser (L2)
# les vecteurs de features : c'est ce qui marche le mieux apparemment.
from pyspark.ml.feature import Normalizer
normalizerUni = Normalizer(inputCol='words',outputCol='normWords',p=2.0)
normalizerBi = Normalizer(inputCol="bigrams",outputCol='normBigrams',p=2.0)
dfNorm = normalizerUni.transform(dfVect2)
dfNorm2 = normalizerBi.transform(dfNorm)
dfNorm2.select('words','normWords').show()
# La différence n'apparait pas dans la table puisqu'on n'a la place de visualiser que les indices des élements 
# non nuls et pas leur valeur

+--------------------+--------------------+
|               words|           normWords|
+--------------------+--------------------+
|(1573,[14,26,28,3...|(1573,[14,26,28,3...|
|(1573,[4,22,25,30...|(1573,[4,22,25,30...|
|(1573,[37,71,85,8...|(1573,[37,71,85,8...|
|(1573,[10,37,61,6...|(1573,[10,37,61,6...|
|(1573,[0,12,20,21...|(1573,[0,12,20,21...|
|(1573,[0,6,16,33,...|(1573,[0,6,16,33,...|
|(1573,[3,23,29,36...|(1573,[3,23,29,36...|
|(1573,[7,9,11,13,...|(1573,[7,9,11,13,...|
|(1573,[17,24,29,3...|(1573,[17,24,29,3...|
|(1573,[8,51,96,13...|(1573,[8,51,96,13...|
|(1573,[4,18,25,33...|(1573,[4,18,25,33...|
|(1573,[1,5,29,64,...|(1573,[1,5,29,64,...|
|(1573,[104,108,11...|(1573,[104,108,11...|
|(1573,[0,2,29,39,...|(1573,[0,2,29,39,...|
+--------------------+--------------------+



In [62]:
# On passe au TFIDF
# Evidemment en choisissant la bonne dataframe parmi celle du dessus, on peut appliquer ces calculs
# à n'importz quelle colonne (bigrammes, avec stop words ou sans...)
from pyspark.ml.feature import HashingTF
htf = HashingTF(inputCol='words',outputCol='wordsTF',numFeatures=10000)
dfTrainTF = htf.transform(dfTrainTokNoSw)

In [67]:
# INverse doc frequency
from pyspark.ml.feature import IDF
idf = IDF(inputCol=htf.getOutputCol(),outputCol="wordsTFIDF")
idfModel = idf.fit(dfTrainTF)
dfTrainTFIDF = idfModel.transform(dfTrainTF)
dfTrainTFIDF.select('review','wordsTF','wordsTFIDF').show()

+--------------------+--------------------+--------------------+
|              review|             wordsTF|          wordsTFIDF|
+--------------------+--------------------+--------------------+
|homelessness (or ...|(10000,[0,14,39,7...|(10000,[0,14,39,7...|
|brilliant over-ac...|(10000,[77,97,105...|(10000,[77,97,105...|
|this is easily th...|(10000,[97,131,51...|(10000,[97,131,51...|
|this is not the t...|(10000,[97,260,41...|(10000,[97,260,41...|
|in this "critical...|(10000,[0,45,82,9...|(10000,[0,45,82,9...|
|the night listene...|(10000,[0,48,97,9...|(10000,[0,48,97,9...|
|you know, robin w...|(10000,[97,105,15...|(10000,[97,105,15...|
|airport '77 start...|(10000,[34,38,47,...|(10000,[34,38,47,...|
|this film lacked ...|(10000,[105,124,4...|(10000,[105,124,4...|
|sorry everyone,,,...|(10000,[49,97,102...|(10000,[49,97,102...|
|the second attemp...|(10000,[45,97,105...|(10000,[45,97,105...|
|i don't know who ...|(10000,[97,105,11...|(10000,[97,105,11...|
|this film is medi...|(10

In [69]:
# Je sais que cette étape m'a été utile une fois, la ça a pas trop l'air
from pyspark.ml.feature import StringIndexer
string_indexer = StringIndexer(inputCol='label', outputCol='target_indexed')
string_indexer_model = string_indexer.fit(dfTrainTFIDF)
dfTrainFinal = string_indexer_model.transform(dfTrainTFIDF)
dfTrainFinal.select('review','label','target_indexed').show()

+--------------------+-----+--------------+
|              review|label|target_indexed|
+--------------------+-----+--------------+
|homelessness (or ...|  1.0|           1.0|
|brilliant over-ac...|  1.0|           1.0|
|this is easily th...|  1.0|           1.0|
|this is not the t...|  1.0|           1.0|
|in this "critical...|  1.0|           1.0|
|the night listene...|  1.0|           1.0|
|you know, robin w...|  1.0|           1.0|
|airport '77 start...|  0.0|           0.0|
|this film lacked ...|  0.0|           0.0|
|sorry everyone,,,...|  0.0|           0.0|
|the second attemp...|  0.0|           0.0|
|i don't know who ...|  0.0|           0.0|
|this film is medi...|  0.0|           0.0|
|the film is bad. ...|  0.0|           0.0|
+--------------------+-----+--------------+



In [70]:
from pyspark.ml.classification import DecisionTreeClassifier
dt = DecisionTreeClassifier(featuresCol=idf.getOutputCol(),labelCol=string_indexer.getOutputCol())
dt_model = dt.fit(dfTrainFinal)

In [71]:
# On applique le même à notre ensemble de test ridicule.
# En théorie le pipeline permet d'automatiser tout ça mais bon, on s'en servira probablement pas

# EDIT : en fait c'est plutot facile de créer des transformers à partir de chaque étape, donc peut 
# être que les pipelines c'est faisables. A voir
df_test_words = tokenizer.transform(dfTest)
df_test_tf = htf.transform(df_test_words)
df_test_tfidf = idfModel.transform(df_test_tf)
df_test_final = string_indexer_model.transform(df_test_tfidf)

In [72]:
# Les prédictions
df_test_pred = dt_model.transform(df_test_final)

In [73]:
df_test_pred.select('review', 'target_indexed', 'prediction', 'probability').show(5)

+--------------------+--------------+----------+-----------+
|              review|target_indexed|prediction|probability|
+--------------------+--------------+----------+-----------+
|bromwell high is ...|           1.0|       0.0|  [1.0,0.0]|
|this isn't the co...|           1.0|       1.0|  [0.0,1.0]|
|yes its an art......|           1.0|       1.0|  [0.0,1.0]|
|story of a man wh...|           0.0|       0.0|  [1.0,0.0]|
|when i was little...|           0.0|       1.0|  [0.0,1.0]|
+--------------------+--------------+----------+-----------+
only showing top 5 rows



In [74]:
# Je fais un pipeline très basique
from pyspark.ml import Pipeline

In [75]:
# Instanciate all the Estimators and Transformers necessary
tokenizer = Tokenizer(inputCol='review', outputCol='reviews_words')
hashing_tf = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol='reviews_tf', numFeatures=10000)
idf = IDF(inputCol=hashing_tf.getOutputCol(), outputCol="reviews_tfidf")
string_indexer = StringIndexer(inputCol='label', outputCol='target_indexed')
dt = DecisionTreeClassifier(featuresCol=idf.getOutputCol(), labelCol=string_indexer.getOutputCol(), maxDepth=10)

In [76]:
# Instanciate a Pipeline
pipeline = Pipeline(stages=[tokenizer,
                            hashing_tf,
                            idf,
                            string_indexer,
                            dt])

In [77]:
pipeline_model = pipeline.fit(dfTrain)
 

In [78]:
df_test_pred = pipeline_model.transform(dfTest)
 

In [79]:
df_test_pred.select('review', 'target_indexed', 'prediction', 'probability').show()

+--------------------+--------------+----------+-----------+
|              review|target_indexed|prediction|probability|
+--------------------+--------------+----------+-----------+
|bromwell high is ...|           1.0|       0.0|  [1.0,0.0]|
|this isn't the co...|           1.0|       1.0|  [0.0,1.0]|
|yes its an art......|           1.0|       1.0|  [0.0,1.0]|
|story of a man wh...|           0.0|       0.0|  [1.0,0.0]|
|when i was little...|           0.0|       1.0|  [0.0,1.0]|
|"it appears that ...|           0.0|       1.0|  [0.0,1.0]|
+--------------------+--------------+----------+-----------+



In [80]:
# Un outil automatique pour calculer le taux de bonne classif.
# La encore pas très utile en vrai
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [81]:
evaluator = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='target_indexed', metricName='precision')


In [82]:
evaluator.evaluate(df_test_pred)

0.5

In [83]:
# La cross validation et le test des différents paramètres du classifieurs c'est pas trop dur sur spark ML, 
# c'est en fait la seule raison pour laquelle cette librairie me paraissait mieux... avec du recul j'aurais 
# perdu moins de temps à recoder moi-même la cross valid et la grid search...

In [84]:
from pyspark.ml.tuning import ParamGridBuilder
from pyspark.ml.tuning import CrossValidator

In [85]:
grid=(ParamGridBuilder()
     .baseOn([evaluator.metricName,'precision'])
     .addGrid(dt.maxDepth, [10,20])
     .build())

In [86]:
cv = CrossValidator(estimator=pipeline, estimatorParamMaps=grid,evaluator=evaluator)

In [87]:
cv_model = cv.fit(dfTrain)

In [88]:
df_test_pred = cv_model.transform(dfTest)

In [89]:
evaluator.evaluate(df_test_pred)

0.5